# 03 — Interactive Dashboard (Plotly)

In [1]:

import sys, subprocess, importlib
def ensure(pkg):
    try:
        importlib.import_module(pkg)
    except Exception:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])
ensure("pandas"); ensure("plotly")

In [2]:
import pandas as pd, plotly.express as px
from pathlib import Path
ROOT = Path.cwd()
candidates = [ROOT.parent / "data/processed/fpds_cleaned.csv", ROOT / "data/processed/fpds_cleaned.csv"]
csv_path = next((p for p in candidates if p.exists()), None)
if csv_path is None:
    p = ROOT.parent / "data/processed"
    p.mkdir(parents=True, exist_ok=True)
    csv_path = p / "fpds_cleaned.csv"
    csv_path.write_text(
        "action_date,year_month,vendor_group,naics_code_clean,dollar_obligated,place_of_performance_state\n"
        "2021-07-15,2021-07,LOCKHEED MARTIN,336413,12000000,TX\n"
        "2021-07-10,2021-07,BOEING,336413,8000000,CA\n"
        "2022-01-02,2022-01,RAYTHEON,334111,2500000,AZ\n",
        encoding="utf-8"
    )
    print(f"[INFO] Created fallback cleaned sample at {csv_path}")
else:
    print(f"[INFO] Using cleaned dataset: {csv_path}")
df = pd.read_csv(csv_path, parse_dates=["action_date"])

[INFO] Using cleaned dataset: c:\Users\taylo\OneDrive\Desktop\DoD-SupplyChain-Disruption-Complete - Copy\data\processed\fpds_cleaned.csv


### Interactive — Contract Volume by Month

In [3]:
ts = df.groupby("year_month").size().reset_index(name="contracts").sort_values("year_month")
fig = px.line(ts, x="year_month", y="contracts", title="Contract Volume Over Time")
fig.update_layout(xaxis_title="Year-Month", yaxis_title="Contracts")
fig.show()

### Interactive — Top Vendors by Count

In [4]:
tv = df["vendor_group"].value_counts().reset_index()
tv.columns = ["vendor_group", "contracts"]
fig = px.bar(tv.head(10), x="vendor_group", y="contracts", title="Top 10 Vendors by Contract Count")
fig.update_layout(xaxis_title="Vendor", yaxis_title="Contracts")
fig.show()

### Interactive — Contract Values Histogram

In [5]:
fig = px.histogram(df, x="dollar_obligated", nbins=30, title="Obligated Values Distribution")
fig.update_layout(xaxis_title="Dollar Obligated", yaxis_title="Frequency")
fig.show()